<a href="https://colab.research.google.com/github/Akshay-ramagiri/NNDL/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MOUNT GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##IMPORTING REQUIRED PACKAGES

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##IMPORT DATA

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Assignment4.csv')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,0:00,5551.82208
1,NaN,1:00,4983.17184
2,NaN,2:00,4888.39680
3,NaN,3:00,5072.95872
4,NaN,4:00,5196.25980


In [ ]:
X = []
Y = []
for i in range(24*7, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-24*7]) 
    Y.append(data['Load (kW)'].iloc[i])

In [ ]:
data = pd.DataFrame({'Previous Day Load' : X, 'Present Day Load' : Y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


##NORMALIZATION

In [ ]:
Xmax = np.max(data['Previous Day Load'])
Xmin = np.min(data['Previous Day Load'])
Ymax = np.max(data['Present Day Load'])
Ymin = np.min(data['Present Day Load'])
data['Previous Day Load'] = (data['Previous Day Load'] - Xmin) / (Xmax - Xmin)
data['Present Day Load'] = (data['Present Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Previous Day Load,Present Day Load
0,0.397516,0.360304
1,0.293377,0.322302
2,0.276021,0.286040
3,0.309820,0.254087
4,0.332401,0.300562


##DATA SPLIT

In [ ]:
x = data['Previous Day Load']
y = data['Present Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)

##GRADIENT CALCULATION AND MODEL TRAINING

In [ ]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.1
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6

for _ in range(epochs):
    for i in range(Xtrain.shape[0]):
        gradM = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c) * Xtrain.iloc[i]
        gradC = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c)
        
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)

print(f'm = {m}\nc = {c}')

m = 0.49185813151358243
c = 0.23533504760803883


In [ ]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

##DENORMALIZATION

In [ ]:
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

##ERROR METRIC CALCULATION
TRAINING DATA

In [ ]:
train_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
train_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
train_RMSE = train_MSE ** 0.5
print('Training Error :')
print(f'MAE : {train_MAE}')
print(f'MSE : {train_MSE}')
print(f'RMSE : {train_RMSE}')

Training Error :
MAE : 736.9777250149242
MSE : 889578.9084822778
RMSE : 943.174908742953


TESTING DATA

In [ ]:
test_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
test_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
test_RMSE = test_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {test_MAE}')
print(f'MSE : {test_MSE}')
print(f'RMSE : {test_RMSE}')

Testing Error :
MAE : 784.1584560870851
MSE : 959946.2709890122
RMSE : 979.768478258518


##Analyze training data predictions

In [ ]:
pd.DataFrame({'Actual' : trainY, 'Predicted':train_pred})

,Actual,Predicted
0,5086.98792,5087.591806
1,6378.60960,5766.995863
2,7647.78456,6223.307441
3,6165.05400,5690.585922
4,5918.45184,5534.390500
...,...,...
1809,5020.11540,5903.858650
1810,4048.82712,5557.021958
1811,5674.03200,5611.721043
1812,5621.65632,7046.939099


##Analyze testing data predictions

In [ ]:
pd.DataFrame({'Actual' : testY, 'Predicted':test_pred})

,Actual,Predicted
0,7609.43808,6804.437267
1,4187.24856,5784.487296
2,6738.69240,6222.693707
3,5640.67368,6247.549953
4,6550.07760,5474.244522
...,...,...
197,5761.79244,6645.249889
198,6955.67736,6127.104563
199,8607.69360,6060.207506
200,6532.93080,6261.128828


##PREDICTION

In [ ]:
x = float(input('Enter the load at previous week same hour : '))
x = (x - Xmin) / (Xmax - Xmin)
pred = m * x + c
pred = (pred * (Ymax - Ymin)) + Ymin
print('The Predicted load at present hour = ', pred)

Enter the load at previous week same hour : 5000
The Predicted load at present hour =  5460.432109953241
